# Baseline

Universal Sentence Encoding + MMR

## 1. Setting

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from nltk.tokenize import sent_tokenize

In [3]:
USB = hub.load("C:/Users/Administrator/Desktop/Project/Universal_Sentence_Encoder_Summarization/universal-sentence-encoder_4")

## 2. Data

In [4]:
data = """
Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.

Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. "Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility," chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.

TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.

"""

In [5]:
sentences = sent_tokenize(data)
sentences

['\nAd sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.',
 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.',
 'TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.',
 'Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.',
 'Time Warner said on Friday that it now owns 8% of search-engine Google.',
 'But its own internet business, AOL, had has mixed fortunes.',
 'It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.',
 "However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.",
 "It hopes to increase subscribers by offering the online service free to TimeWarner internet c

In [6]:
embeddings = USB(sentences).numpy()

In [7]:
embeddings.shape

(20, 512)

## 3. Model

In [52]:

class MMR(object):
    
    def get_query(self, embeddings):
        query = np.mean(embeddings, axis=0)
        return query
    
    def similarity(self, s1, s2):
        cossim = np.dot(s1, s2) / (np.linalg.norm(s1) * np.linalg.norm(s2))
        sim = 1 - np.arccos(cossim) / np.pi
        return sim
    
    def similarity_with_matrix(self, s, m):
        cossim = np.dot(m, s) / (np.linalg.norm(s) * np.linalg.norm(m, axis=1))
        sim = 1 - np.arccos(cossim) / np.pi
        return sim
    
    def mmr_score(self, s, q, selected, _lambda):
        relevance = _lambda * self.similarity(s, q)
        if selected.shape[0] > 0:
            diversity = (1 - _lambda) * np.max(self.similarity_with_matrix(s, selected))
        else:
            diversity = 0
        return relevance - diversity
            
        
    def summarize(self, embeddings, max_len=3, _lambda=0.5):
        query = self.get_query(embeddings)
        
        selected = [False] * embeddings.shape[0]
        while sum(selected) < max_len:
            remains = [idx for idx, i in enumerate(selected) if not i]
            mmr = [self.mmr_score(embeddings[i], query, embeddings[selected], _lambda) for i in remains]
            best = np.argsort(mmr)[-1]
            selected[best] = True
        
        return selected

In [53]:
mmr = MMR()

In [54]:
mmr.summarize(embeddings)

[False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

In [45]:
query = mmr.get_query(embeddings)
selected = [False] * embeddings.shape[0]
remains = [i for i in range(len(embeddings)) if not selected[i]]

In [21]:
selected[0] = True
selected[1] = True

In [46]:
m = embeddings[selected]

In [48]:
m.shape[0]

0

In [35]:
np.max(similarity_with_matrix(query, m))

0.7192533

In [24]:
np.dot(embeddings[selected], query)

array([0.3185968, 0.279523 ], dtype=float32)

In [27]:
np.linalg.norm(embeddings[selected], axis=1)

array([0.99999994, 0.9999999 ], dtype=float32)

In [22]:
embeddings[selected].shape

(2, 512)

In [17]:
remains

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [15]:
selected

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [13]:
query.shape

(512,)

In [10]:
mmr.summarize(embeddings)

ValueError: shapes (512,) and (0,20,512) not aligned: 512 (dim 0) != 20 (dim 1)